# Crime data pre-proccessing 

## 



In [2]:
# import libraries

import pandas as pd
import numpy as np
import utm
import re

In [3]:
# read csv

data = pd.read_csv('crimedata_csv_AllNeighbourhoods_2021+2022.csv')


In [4]:
# create copy for safety

df = data.copy()

In [47]:
# dataframe

df

,TYPE,YEAR,MONTH,DAY,HOUR,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y,datetime,lat,long,weekday,weekday_date,time_of_week,season
0,Break and Enter Commercial,2021,11,21,6,10XX ALBERNI ST,West End,491015.9434,5459166.140,2021-11-21 06:00:00,49.285181,-123.123536,Sunday,6,Weekend,November - April (Colder Months)
1,Break and Enter Commercial,2021,11,26,13,10XX BARCLAY ST,West End,490833.8455,5458886.535,2021-11-26 13:00:00,49.282663,-123.126034,Friday,4,Weekday,November - April (Colder Months)
2,Break and Enter Commercial,2021,12,14,2,10XX BEACH AVE,Central Business District,490252.3815,5458162.723,2021-12-14 02:00:00,49.276144,-123.134011,Tuesday,1,Weekday,November - April (Colder Months)
3,Break and Enter Commercial,2021,7,17,5,10XX BEACH AVE,Central Business District,490255.3982,5458158.788,2021-07-17 05:00:00,49.276108,-123.133970,Saturday,5,Weekend,May - October (Warmer Months)
4,Break and Enter Commercial,2021,6,15,4,10XX BEACH AVE,Central Business District,490258.4148,5458154.853,2021-06-15 04:00:00,49.276073,-123.133928,Tuesday,1,Weekday,May - October (Warmer Months)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35454,Vehicle Collision or Pedestrian Struck (with I...,2022,1,7,14,W 7TH AVE / ALDER ST,Fairview,490448.0000,5456949.000,2022-01-07 14:00:00,49.265229,-123.131293,Friday,4,Weekday,November - April (Colder Months)
35455,Vehicle Collision or Pedestrian Struck (with I...,2022,1,18,20,WILLOW ST / W BROADWAY AVE,Fairview,491136.0000,5456735.000,2022-01-18 20:00:00,49.263314,-123.121832,Tuesday,1,Weekday,November - April (Colder Months)
35456,Vehicle Collision or Pedestrian Struck (with I...,2022,2,8,19,WINDSOR ST / E 49TH AVE,Sunset,493792.0000,5452516.000,2022-02-08 19:00:00,49.225396,-123.085261,Tuesday,1,Weekday,November - April (Colder Months)
35457,Vehicle Collision or Pedestrian Struck (with I...,2022,1,4,14,X BLOCK E 33RD AVE,Riley Park,492354.0000,5454205.000,2022-01-04 14:00:00,49.240573,-123.105042,Tuesday,1,Weekday,November - April (Colder Months)


In [6]:
# find number of unique values

df.nunique()

TYPE                11
YEAR                 2
MONTH               12
DAY                 31
HOUR                24
MINUTE              60
HUNDRED_BLOCK     7901
NEIGHBOURHOOD       24
X                15691
Y                15587
dtype: int64

## Cleaning data

In [7]:
#  for certain crimes like homicides, some of the data is unlabelled in order to respect the privacy of victims. 

df.isnull().sum()

TYPE             0
YEAR             0
MONTH            0
DAY              0
HOUR             0
MINUTE           0
HUNDRED_BLOCK    0
NEIGHBOURHOOD    9
X                2
Y                2
dtype: int64

In [8]:
# creating column with datetime object

dt = df.columns[1:5]
df['datetime'] = pd.to_datetime(df[dt])


In [9]:
# This is not important, recorded time the crime occcured does not accurately reflect the actual time the crime occurred
# There is always a slight delay in reporting of the crime

df = df.drop(columns=['MINUTE'])


In [10]:
# count of types of crime in 2021 and 2022

df[['YEAR', 'TYPE']].groupby(['TYPE']).count()


,YEAR
TYPE,
Break and Enter Commercial,2322
Break and Enter Residential/Other,1649
Homicide,18
Mischief,5996
Offence Against a Person,4117
Other Theft,9595
Theft from Vehicle,8212
Theft of Bicycle,1508
Theft of Vehicle,927


In [11]:
# there is a poor correlation between any of these variables

df.corr()

,YEAR,MONTH,DAY,HOUR,X,Y
YEAR,1.000000,-0.434988,-0.065721,0.012028,0.000793,0.000612
MONTH,-0.434988,1.000000,0.023954,-0.002784,-0.001674,-0.001526
DAY,-0.065721,0.023954,1.000000,0.002544,-0.011597,-0.011679
HOUR,0.012028,-0.002784,0.002544,1.000000,0.537629,0.537564
X,0.000793,-0.001674,-0.011597,0.537629,1.000000,0.999877
Y,0.000612,-0.001526,-0.011679,0.537564,0.999877,1.000000


In [12]:
# dropping null UTM-coordinates in the X and Y columns
# I decided to drop this they wouldn't work with my tableau dashboard

df=df.dropna(subset=['X', 'Y'])


In [13]:
# The lattitude and longitude in the database are in UTM coordinate system
# In order to create a tableau dashboard, the UTM coordiantes need to be converted to lat/long WGS84 coordinate

# this function takes the UTM coordinates in that region and turns them into WGS84 coordinates
def rule(row):
    try:
        lat, long = utm.to_latlon(easting=row["X"], northing=row["Y"], zone_number=10, zone_letter='N')
    except:
        lat = None
        long = None
        pd.Series({"lat": lat, "long": long})
    return pd.Series({"lat": lat, "long": long})


In [14]:
# run function

df=df.merge(df.apply(rule, axis=1), left_index=True, right_index=True)


In [15]:
# extract weekday from datetime object

# this could be useful to know as there could be more thefts during certain days of the week (like the weekend)
# thefts could also be less likely to occur during specific days when at-risk populations recieve income and disability assistance
# This is only speculation and would require a proper study to make any explicit claims. What I am doing here can be seen as pre-study research

# day name
df['weekday']=df['datetime'].dt.day_name()

# day number
df['weekday_date']=df['datetime'].dt.weekday

In [16]:
'''Upon further expirmentation in tableau I found the best way to feature-engineer the data by weekdays, while still having enough data points is to split the days by weekend and weekday. If the user decides they want all the data points they can still choose the "All" button'''

'Upon further expirmentation in tableau I found the best way to feature-engineer the data by weekdays, while still having enough data points is to split the days by weekend and weekday. If the user decides they want all the data points they can still choose the "All" button'

In [17]:
# function that converts the value from 'weekday_date' column and turns it 'weekend' or 'weekday'

def time_of_week_func(x):
    if x>4 or x==0:
        w='Weekend'
    else:
        w='Weekday'
    return w

In [18]:
# run time_of_week_func function

df['time_of_week']=df['weekday_date'].apply(time_of_week_func)

In [19]:
# turning months into seasons

# I found the best way to have enough data points while still having relevent information is by splitting the months into warmer and colder months

# label numbered month into warmer/colder season
def season_func(row):
    if row['MONTH'] >=5 and row['MONTH'] <=10:
        season='May - October (Warmer Months)'
    else:
        season='November - April (Colder Months)'
    return season


In [20]:
# run function
df['season']=df.apply(season_func, axis=1, raw=False)

In [21]:
#this is our new dataset

df.head(5)

,TYPE,YEAR,MONTH,DAY,HOUR,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y,datetime,lat,long,weekday,weekday_date,time_of_week,season
0,Break and Enter Commercial,2021,11,21,6,10XX ALBERNI ST,West End,491015.9434,5459166.140,2021-11-21 06:00:00,49.285181,-123.123536,Sunday,6,Weekend,November - April (Colder Months)
1,Break and Enter Commercial,2021,11,26,13,10XX BARCLAY ST,West End,490833.8455,5458886.535,2021-11-26 13:00:00,49.282663,-123.126034,Friday,4,Weekday,November - April (Colder Months)
2,Break and Enter Commercial,2021,12,14,2,10XX BEACH AVE,Central Business District,490252.3815,5458162.723,2021-12-14 02:00:00,49.276144,-123.134011,Tuesday,1,Weekday,November - April (Colder Months)
3,Break and Enter Commercial,2021,7,17,5,10XX BEACH AVE,Central Business District,490255.3982,5458158.788,2021-07-17 05:00:00,49.276108,-123.133970,Saturday,5,Weekend,May - October (Warmer Months)
4,Break and Enter Commercial,2021,6,15,4,10XX BEACH AVE,Central Business District,490258.4148,5458154.853,2021-06-15 04:00:00,49.276073,-123.133928,Tuesday,1,Weekday,May - October (Warmer Months)


# Thefts from Vehicles Project

For the next part of the project I created a map which includes the thefts from vehicles.

Value statement: As someone who frequently parks their car in Vancouver, it always is a bit nerve-racking leaving my car parked on the street/in a parking lot, as I may come back to find a window smashed and the contents in my car stolen. I wanted to create a dashboard which could visualize where and when these thefts occur, so that the user could chooose safer spots to park their vehicle.

In [60]:
# first I will drop all the other types of crimes

df_car_thefts=df[df['TYPE']=='Theft from Vehicle']

In [61]:
# occurences of breakins in 2021 and 2022 by neighbourhood

# creat filter for previous year (its the only full year you can get real data from)
filt=df_car_thefts['YEAR']==2021

# groupby+filt
df_car_thefts[filt][['NEIGHBOURHOOD', 'TYPE']].groupby(['NEIGHBOURHOOD']).count()

,TYPE
NEIGHBOURHOOD,
Arbutus Ridge,78
Central Business District,1947
Dunbar-Southlands,90
Fairview,374
Grandview-Woodland,364
Hastings-Sunrise,385
Kensington-Cedar Cottage,400
Kerrisdale,135
Killarney,132


In [62]:
# count of breakings during each day
# we can see that break-ins happen more often during the weekend and monday and decreases during the week

# groupby+filt
df_car_thefts[filt][['weekday', 'TYPE']].groupby(['weekday']).count()

,TYPE
weekday,
Friday,1205
Monday,1094
Saturday,1128
Sunday,1111
Thursday,968
Tuesday,990
Wednesday,979


In [63]:
# how many breakings happen every month

# groupby+filt
df_car_thefts[filt][['MONTH', 'TYPE']].groupby(['MONTH']).count()

,TYPE
MONTH,
1,711
2,565
3,546
4,525
5,600
6,559
7,689
8,732
9,714


In [64]:
# how many car thefts happen during the colder/warmer months

# groupby+filt
df_car_thefts[filt][['season', 'TYPE']].groupby(['season']).count()

,TYPE
season,
May - October (Warmer Months),3920
November - April (Colder Months),3555


In [65]:
# drop the columns which are not needed
df_car_thefts=df_car_thefts.drop(columns=['TYPE','X','Y', 'datetime','weekday_date'])

In [66]:
# dataset

df_car_thefts

,YEAR,MONTH,DAY,HOUR,HUNDRED_BLOCK,NEIGHBOURHOOD,lat,long,weekday,time_of_week,season
21218,2021,10,28,17,0X KEEFER ST,Central Business District,49.279185,-123.104351,Thursday,Weekday,May - October (Warmer Months)
21219,2021,12,13,17,10XX ALBERNI ST,West End,49.284871,-123.123063,Monday,Weekend,November - April (Colder Months)
21220,2021,2,4,17,10XX ALBERNI ST,West End,49.284981,-123.123053,Thursday,Weekday,November - April (Colder Months)
21221,2021,2,15,14,10XX ALBERNI ST,West End,49.284794,-123.122946,Monday,Weekend,November - April (Colder Months)
21222,2021,4,11,13,10XX ALBERNI ST,West End,49.284794,-123.122946,Sunday,Weekend,November - April (Colder Months)
...,...,...,...,...,...,...,...,...,...,...,...
35223,2022,1,20,16,X E CORDOVA ST,Central Business District,49.282458,-123.104155,Thursday,Weekday,November - April (Colder Months)
35224,2022,1,3,21,X NK_LOC ST,Strathcona,49.281851,-123.099466,Monday,Weekend,November - April (Colder Months)
35225,2022,1,25,21,X NK_LOC ST,Strathcona,49.281851,-123.099466,Tuesday,Weekday,November - April (Colder Months)
35226,2022,2,2,14,X NK_LOC ST,Strathcona,49.281851,-123.099466,Wednesday,Weekday,November - April (Colder Months)


## More car-thefts pre-processsing

In [67]:
# dropping the months of of 2021 which overlap with 2022 to get rid of garbage data

'''Including the same months in 2021 and 2022 will skew the data and make it appear that more car breakins are happening during that month.
Although more data points are generally a good thing, it will be bad data if they are included in the data-set.
Any sort of exploratory analysis, or statistical analysis will get spoiled if we use the bad data
As the old adage goes garbage in, garbage out
Therefore the overlapping months need to get deleted removed from the previous year'''


# first we need to find what the greatest (most recent) month is in 2022

# create filter for 2022
year_filt=df_car_thefts['YEAR']==2022


# create filter to find the most recent month in 2022
month_max=df_car_thefts[year_filt]['MONTH'].max()
month_filt=df_car_thefts['MONTH']==month_max


# It would be bad practice to include a month that has only partially passed, however I still wanted to include it if the month had mostly passed. 
# I comprimised by chooseing to include a  month in the data only if at least 24 days (around 80% of the month) had passed

#find the greatest day for the most recent month
day_max=df_car_thefts[month_filt][year_filt]['DAY'].max()

# function which finds out whether approx. 80% of the month has passed
def month_func(x):
    if x>=24:
        ret=month_max
    else:
        ret=month_max-1
    return ret

# run function
actual_month_max=month_func(day_max)
actual_month_max #this is the month which will be 

C:\Users\lazar\AppData\Local\Temp/ipykernel_14168/763320558.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  day_max=df_car_thefts[month_filt][year_filt]['DAY'].max()


1

In [54]:
df_car_thefts[filt]

,YEAR,MONTH,DAY,HOUR,HUNDRED_BLOCK,NEIGHBOURHOOD,lat,long,weekday,time_of_week,season
21218,2021,10,28,17,0X KEEFER ST,Central Business District,49.279185,-123.104351,Thursday,Weekday,May - October (Warmer Months)
21219,2021,12,13,17,10XX ALBERNI ST,West End,49.284871,-123.123063,Monday,Weekend,November - April (Colder Months)
21220,2021,2,4,17,10XX ALBERNI ST,West End,49.284981,-123.123053,Thursday,Weekday,November - April (Colder Months)
21221,2021,2,15,14,10XX ALBERNI ST,West End,49.284794,-123.122946,Monday,Weekend,November - April (Colder Months)
21222,2021,4,11,13,10XX ALBERNI ST,West End,49.284794,-123.122946,Sunday,Weekend,November - April (Colder Months)
...,...,...,...,...,...,...,...,...,...,...,...
28688,2021,11,23,18,X NK_LOC ST,Strathcona,49.281851,-123.099466,Tuesday,Weekday,November - April (Colder Months)
28689,2021,12,5,15,X NK_LOC ST,Strathcona,49.281851,-123.099466,Sunday,Weekend,November - April (Colder Months)
28690,2021,9,17,12,X W 23RD AVE,Riley Park,49.249880,-123.105230,Friday,Weekday,May - October (Warmer Months)
28691,2021,6,14,0,X W 38TH AVE,Riley Park,49.236464,-123.105739,Monday,Weekend,May - October (Warmer Months)


In [50]:
df_car_thefts[filt]['MONTH']!=actual_month_max

21218    True
21219    True
21220    True
21221    True
21222    True
         ... 
28688    True
28689    True
28690    True
28691    True
28692    True
Name: MONTH, Length: 7475, dtype: bool

In [59]:
actual_month_max

1

In [72]:
# a filter which gets rid of the month(s) in the previous year

sel_rows=df_car_thefts[(df_car_thefts['MONTH']==actual_month_max) & (df_car_thefts['YEAR']==2021)].index
df_car_thefts=df_car_thefts.drop(sel_rows, axis=0)

# dataframe
df_car_thefts.head()

,YEAR,MONTH,DAY,HOUR,HUNDRED_BLOCK,NEIGHBOURHOOD,lat,long,weekday,time_of_week,season
21218,2021,10,28,17,0X KEEFER ST,Central Business District,49.279185,-123.104351,Thursday,Weekday,May - October (Warmer Months)
21219,2021,12,13,17,10XX ALBERNI ST,West End,49.284871,-123.123063,Monday,Weekend,November - April (Colder Months)
21220,2021,2,4,17,10XX ALBERNI ST,West End,49.284981,-123.123053,Thursday,Weekday,November - April (Colder Months)
21221,2021,2,15,14,10XX ALBERNI ST,West End,49.284794,-123.122946,Monday,Weekend,November - April (Colder Months)
21222,2021,4,11,13,10XX ALBERNI ST,West End,49.284794,-123.122946,Sunday,Weekend,November - April (Colder Months)


# Parking meter data set

In [ ]:
# load parking meter data set

'''this is a dataset I found which includes the coordinates of all city pay parkins'''

park_df=pd.read_csv('parking-meters.csv', ';')
park_df.head()

C:\Users\lazar\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,METERHEAD,R_MF_9A_6P,R_MF_6P_10,R_SA_9A_6P,R_SA_6P_10,R_SU_9A_6P,R_SU_6P_10,RATE_MISC,TIMEINEFFE,T_MF_9A_6P,...,T_SA_9A_6P,T_SA_6P_10,T_SU_9A_6P,T_SU_6P_10,TIME_MISC,CREDITCARD,PAY_PHONE,Geom,Geo Local Area,METERID
0,Twin,$1.00,$1.00,$1.00,$1.00,$1.00,$1.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,3 Hr,...,3 Hr,4 Hr,3 Hr,4 Hr,NaN,No,56533,"{""coordinates"": [-123.03362137823906, 49.23285...",Killarney,993358
1,Twin,$1.00,$1.00,$1.00,$1.00,$1.00,$1.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,3 Hr,...,3 Hr,4 Hr,3 Hr,4 Hr,NaN,No,56469,"{""coordinates"": [-123.03313623855607, 49.23286...",Renfrew-Collingwood,993371
2,Twin,$1.00,$1.00,$1.00,$1.00,$1.00,$1.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,2 Hr,...,2 Hr,4 Hr,2 Hr,4 Hr,NaN,No,57554,"{""coordinates"": [-123.10096093278823, 49.25907...",Mount Pleasant,512904
3,Twin,$1.00,$4.00,$1.00,$4.00,$1.00,$4.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,2 Hr,...,2 Hr,4 Hr,2 Hr,4 Hr,NaN,No,56215,"{""coordinates"": [-123.10131362071866, 49.25781...",Mount Pleasant,513019
4,Twin,$1.00,$4.00,$1.00,$4.00,$1.00,$4.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,2 Hr,...,2 Hr,4 Hr,2 Hr,4 Hr,NaN,No,57150,"{""coordinates"": [-123.10103668736426, 49.25781...",Mount Pleasant,513018


In [ ]:
# only need the longitude/lattitude column

park_df=park_df[['Geom']]

In [ ]:
# number of long/lat points on the graph

park_df.nunique()

Geom    4715
dtype: int64

In [ ]:
# dropping duplicates

park_df=park_df.drop_duplicates(subset=['Geom'], keep=False)

In [ ]:
# null count

park_df.isnull().sum()

Geom    1
dtype: int64

In [ ]:
# drop the nulls

park_df=park_df.dropna()

In [ ]:
# pre-processing longitude/lattitude vales

# the data value describing the data for the long and lat is messy and requires some data cleaning
# the data is stored as a set of dictionaries. Pandas cannot interpret dictionaries, so they must be turned into a string and pre-processed further

# turn data into string to prepare it for pre-processing
park_df=park_df[['Geom']].astype(str)

# function to extract longitude from data
def long_func(x):
    a=x.split()
    if len(a)>1:
        ret=a[1]
        ret = re.sub(r'[^-\d*\.\d*]', "", ret)
    else:
        ret=None
    return ret

# function to extract lattitude from data
def lat_func(x):
    a=x.split()
    if len(a)>1:
        ret=a[2]
        ret = re.sub(r'[^-\d*\.\d*]', "", ret)
    else:
        ret=None
    return ret

In [ ]:
# run functions

park_df['park_lat']=[lat_func(x) for x in park_df['Geom']]
park_df['park_long']=[long_func(x) for x in park_df['Geom']]

# dataframe
park_df.head()

,Geom,park_long,park_lat
5,"{""coordinates"": [-123.10249423430041, 49.27706...",-123.10249423430041,49.27706523628811
6,"{""coordinates"": [-123.1029445090042, 49.277403...",-123.1029445090042,49.27740316288875
7,"{""coordinates"": [-123.0898869035528, 49.270338...",-123.0898869035528,49.27033821755311
8,"{""coordinates"": [-123.10556622620037, 49.27691...",-123.10556622620037,49.27691853858081
9,"{""coordinates"": [-123.10376713040408, 49.27681...",-123.10376713040408,49.276811605350154


In [ ]:
# drop the columns that we don't need

park_df=park_df.drop(columns=['Geom'])

In [ ]:
# turns each value individually into int
park_df=park_df.applymap(lambda x: float(x))

#check that it's correct
park_df.dtypes

park_long    float64
park_lat     float64
dtype: object

In [ ]:
# dataframe

park_df.head()

,park_long,park_lat
5,-123.102494,49.277065
6,-123.102945,49.277403
7,-123.089887,49.270338
8,-123.105566,49.276919
9,-123.103767,49.276812


# Merge and export

In [ ]:
# join both data frames
# i am doing this as it might work in tableau not sure

# reset index
df_car_thefts=df_car_thefts.reset_index(drop=True)
park_df=park_df.reset_index(drop=True)

#merge on index
combo_df=df_car_thefts.join(park_df)


In [936]:
combo_df.isnull().sum()

YEAR                0
MONTH               0
DAY                 0
HOUR                0
HUNDRED_BLOCK       0
NEIGHBOURHOOD       0
lat                 0
long                0
weekday             0
time_of_week        0
season              0
park_long        6043
park_lat         6043
dtype: int64

In [941]:
# export the data frames

df.to_excel('all_crimes_dataset.xlsx')
park_df.to_excel('city_parking_dataset.xlsx')
combo_df.to_excel('car_thefts_and_parking_dataset.xlsx')
df_car_thefts.to_excel('car_thefts_dataset.xlsx')